<a href="https://colab.research.google.com/github/ktamaral/model_vector_comparison/blob/main/token_cost_estimator_cohere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script estimates the average number of tokens per document using Cohere's tokenizer, based on a single representative text block. The output is a variable assignment (average_tokens_per_doc) printed to the console.

**Important: The doc_text must represent an average document text content including all content chunks to ensure accurate cost estimates.**

The transformed dev data is stored in the project Google share drive in the Elasticsearch Consulting directory.

https://drive.google.com/drive/u/0/folders/1Sl0yU2G4jL2SVtpp6SgG9j1UqMxPu1_r

To generate a new transformed "content" block for the record, run the record in the Airflow ingest pipeline and copy the text output from S3 storage JSON object.

Example finding aids:

*   Boston Women's Community Radio sch01379 https://hollisarchives.lib.harvard.edu/repositories/8/resources/6145
*   John Reed Papers hou00070 https://hollisarchives.lib.harvard.edu/repositories/24/resources/1316

Document counts:
https://docs.google.com/spreadsheets/d/1qvUSduwNdAZvzNtjpIexvtAhh5Jb3jFDK6r3YgtqRks/edit?gid=0#gid=0&range=H4

We got the count of records in LibraryCloud using https://api.lib.harvard.edu/v2/items?source=MH:OASIS. Which gave a total of 2,843,879. (it's a bit higher now if you use the same call).

~2.9 million in the spreadsheet, about 3.1 million in aspace searching for everything: https://hollisarchives.lib.harvard.edu/search?utf8=%E2%9C%93&op%5B%5D=&q%5B%5D=*&field%5B%5D=&commit=Search++&limit=&from_year%5B%5D=&to_year%5B%5D=

Finding aid totals: 3.1 million
Total for everything: 30 million


In [4]:
!pip install cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.2/259.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 34.3 MB/s eta 0:00:00


In [7]:
import cohere
from google.colab import userdata

api_key = userdata.get("COHERE_API_KEY")

co = cohere.Client(api_key)

# Content
# Paste in a representative text block (one average document's full content including all chunks)

# Content Example 1
# Boston Women's Community Radio sch01379 https://hollisarchives.lib.harvard.edu/repositories/8/resources/6145
doc_text = """
Boston Women's Community Radio. Records of Boston Women's Community Radio, 1972-1993 (inclusive), 1980-1989 (bulk): A Finding Aid Boston Women's Community Radio Bolinsky, Eileen Boston Women's Community Radio 1972-1993 Audio recordings, phone logs and programming books, promotional material, budgets, flyers, etc. of Boston Women's Community Radio. Copyright. Copyright in the records created by Boston Women's Community Radio are held by the President and Fellows of Harvard College for the Schlesinger Library. Copyright in other audiotapes in the collection may be held by their authors, or the authors' heirs or assigns.\n\nCopying. Materials may be copied in accordance with the library's usual procedures. Boston Women's Community Radio (BWCR) was founded in 1977, and incorporated as a non-profit in 1985. Beginning in 1978, BWCR's core staff and volunteers produced twenty-four hours of programming in commemoration of International Women's Day (IWD), which aired annually in March on WMBR-FM, student radio station licensed to the Massachusetts Institute of Technology, in Cambridge, Massachusetts. BWCR served as an umbrella organization and training ground for feminist radio producers and local community groups, with many of the programs produced for the yearly event distributed afterwards to national audiences. The organization aimed for high-quality programming that was entertaining as well as informative, both political and cultural in content. Shows included sensitive issues such as sexual abuse, lesbian equality, lesbian parenting, children of divorce, Palestinian rights, women organizing in Central America; and they offered a musical blend of Latin American, African, jazz, rock, folk and women's music all highlighting women performers. Moreover, the yearly event featured women writers, playwrights, and poets, including work coming out of Jefferson Park Writing Center (and housing project) in Cambridge. The collection is arranged in five series: The collection of audiotapes and related materials document roughly twenty years of BWCR activities, the bulk of which falls between 1980 and 1989. The archivist created the folder titles and arrangement of the papers, while leaving audiotapes in chronological order, or in the numerical order utilized by the organization. \n\n Series I, LIVE OFF-AIR BROADCAST RECORDINGS, 1982-1989 (#T-317.1 - T-317.80), contains the live off-air recordings, made in-house, of each year's event. IWD programming started at 12 am on a Saturday night, ending around midnight the following day. Years prior to 1982, 1983 and 1984, and after 1989, were not included with the collection. There are gaps in 1985 and 1987 recordings. For yearly scheduled line-ups, see folders #1.1 (1982-1983) and #1.13 (1972, 1980-1992). For further information on various IWD \"greetings,\" see Series II, Suberies B. Titles are derived from the tape labels, with some additional puncuation and italicization added for clarity in the descriptions. Subseries A-F are arranged chronologically. Format is audiocassette. \n\nSeries II, PRE-RECORDED PROGRAMS AND PROMOTIONAL PIECES, INTERNATIONAL GREETINGS AND MUSICAL INTERLUDES, 1972, 1980-1992, undated (#T-317.81 - T-317.224), contains the pre-recorded shows in their entirety, supplimental material used to promote IWD programming, and musical segues. \n\nSubseries A, Pre-recorded programs and promotional pieces, 1972, 1980-1992, undated (#T-317.81 - T-317.176), contains complete programs or promotional inserts produced either with assistance by BWCR, during the event (call-in shows), or acquired for the event; and includes material from the years prior to 1982, the year off-air recording of the events began, and after 1989, the year it ended. Titles are derived from the tape labels, with some additional punctuation for single programs, and italicization added for nationally distributed programs. \n\nSubseries B, IWD greetings and musical interludes, undated (#T-317.177 - T-317.224), consists of brief celebratory greetings from women all over the world in honor of IWD; as well as musical interludes used throughout the days's programming. Tapes are listed in original order. Format is cart tape. \n\nSeries III, ADMINISTRATIVE, 1981-1993 (#1.1-1.12), includes broadcast schedules, production and engineering guidelines, assorted scripts, meetings notes, and phone logs noting listeners' responses.\n\nSeries IV, BROADCAST-RELATED MATERIALS, 1972-1992 (#1.13-1.22), includes broadcast schedules, production and engineering guidelines, assorted scripts, meetings notes, and phone logs noting listeners' responses.\n\nSeries V, PHOTOGRAPHS AND OVERSIZED, 1987-1989 (#PD.1, F+D.1), contains photographs from a 1987 shoot in the studio, and large-format posters advertising the year's line-up, including one mock-up.\n\n Most of the photographs in this collection are or will be cataloged in VIA, Harvard University's Visual Information Access database. AIDS (Disease) Audiotapes Children's radio programs Feminism--United States Feminists--Massachusetts Feminist music Folk music Gospel music Indian women--United States International Women's Day Interviews Israel--Foreign relations--Palestine Jazz Lesbians Music--Latin America Palestine--Foreign relations--Israel Photographs Poetry Posters. Promotional announcements Radio and women--Massachusetts--Boston Radio programs Reggae music Sexual abuse victims Women in radio broadcasting--Massachusetts--Boston Women radio producers and directors--Massachusetts--Boston Women rock musicians Women singers Women's music Women with disabilities World music
"""

# Content Example 2
# John Reed Papers hou00070 https://hollisarchives.lib.harvard.edu/repositories/24/resources/1316
# doc_text = """
# John Reed papers Reed, John, 1887-1920 Reed, John, 1887-1920 1903-1967 The papers of John Reed, the American journalist and revolutionary, including papers relating to his wife, writer Louise Bryant. Images linked to this finding aid are intended for public access and educational use. This material is owned and/or held by the Houghton Library, and is provided solely for the purpose of teaching or individual research. Any other use, including commercial reuse, mounting on other systems, or other forms of redistribution requires the permission of the curator. John Silas Reed was an American poet, journalist, revolutionary writer, and political activist. He graduated from Harvard College in 1910 and soon joined the staff of "The Masses", a Socialist newspaper. He was a war correspondent in Mexico and Europe for "Metropolitan Magazine" and a publicist for the Russian Revolution. During his years in New York City, he became part of the literary and radical circle of intellectuals surrounding Max Eastman, Emma Goldman, and others, and during this period he married Louise Bryant, a fellow journalist and writer. He was frequently arrested for organizing and defending labor strikes, and he became known as a radical leader who went on to help form the American Communist Party. He is most famous for his book, "Ten Days that Shook the World" (1919), where he recorded his eyewitness account of the 1917 Bolshevik Revolution in Russia. He became head of the U.S. Communist Labor Party in 1919, was indicted for treason, and escaped to the Soviet Union where he continued working with Lenin and others in the Soviet government. He died of typhus in 1920 and was buried beside the Kremlin wall. Arranged into the following series: Please note that item numbers 433, 551, 960, and 1149 were inadvertently omitted from this finding aid. The papers of John Silas Reed, poet, journalist, and revolutionary, include his correspondence, personal miscellany, letters to and from his wife (Louise Bryant), notes, and drafts and final manuscripts for his writings. His writings include: <genreform>poetry</genreform>, songs, prose, books, <genreform>pamphlets, </genreform> <genreform>plays</genreform>, newspaper reports, speeches, and revolutionary writings. There are letters and compositions relating to his family and his diverse group of friends and colleagues such as: William Jennings Bryan, Louise Bryant, Charles Townsend Copeland, Max and Crystal Eastman, Emma Goldman, Robert Canby Hallowell, Morris Hillquit, Carl Hovey, Edward Eyre Hunt, Vladimir Il\'ich Lenin, Walter Lippman, Eugene O\'Neill, Boardman Robinson, Alan Seeger, Upton Sinclair, Lincoln Steffens, Julian Street, and many others. One section of particular note is the photographs that record Reed\'s journalistic days covering Francisco Villa in Mexico, the 1912 Industrial Workers of the World strike in Paterson, New Jersey, the 1914 United Mine Workers\' strike in Ludlow, Colorado, and the voluminous section of photographs of the Russian Revolution. There are materials throughout documenting radical left-wing movements in the United States for the period ca.1912-1920, particularly the extensive pamphlet, magazine, and newspaper sections. Reed\'s years in Russia are extensively chronicled in his notes for "Ten Days that Shook the World", his many Russian <genreform>notebooks</genreform> and <genreform>scrapbooks</genreform>, and the series titled, Documents of the Russian revolution. Much of the printed material in Russian in this collection was given to the New York Public Library. The Houghton Library has other John Reed accessions. The reader should check HOLLIS for additional material.'
# """

# Tokenize the text
response = co.tokenize(text=doc_text, model="embed-multilingual-v3.0")
tokens = response.tokens
average_tokens_per_doc_calculated = len(tokens)

# Output the result
print(f"average_tokens_per_doc_calculated = {average_tokens_per_doc_calculated}")

average_tokens_per_doc_calculated = 1371


In [8]:
# Average tokens per document calculated by the tokenizer above
average_tokens_per_doc = average_tokens_per_doc_calculated

# Set document counts
# See column G https://docs.google.com/spreadsheets/d/1qvUSduwNdAZvzNtjpIexvtAhh5Jb3jFDK6r3YgtqRks/edit?gid=0#gid=0
num_documents_finding_aids_only = 3_050_000
num_documents_total = 31_600_000
# https://cohere.com/pricing
price_per_million_tokens = 0.10

# Calculate cost
def calculate_cost(num_docs, avg_tokens, price_per_million):
    total_tokens = num_docs * avg_tokens
    cost = (total_tokens / 1_000_000) * price_per_million
    return int(total_tokens), round(cost, 2)

# Calculate dev set and full data set
finding_aid_tokens, finding_aid_cost = calculate_cost(num_documents_finding_aids_only, average_tokens_per_doc, price_per_million_tokens)
full_tokens, full_cost = calculate_cost(num_documents_total, average_tokens_per_doc, price_per_million_tokens)

# Print results
print("Embedding Cost Estimate:")
print("------------------------")
print(f"Average tokens per document: {average_tokens_per_doc}")
print()
print(f"Finding Aids Only:")
print(f"  Documents: {num_documents_finding_aids_only:,}")
print(f"  Estimated Tokens: {finding_aid_tokens:,}")
print(f"  Estimated Cost: ${finding_aid_cost:.2f}")
print()
print(f"Full Set:")
print(f"  Documents: {num_documents_total:,}")
print(f"  Estimated Tokens: {full_tokens:,}")
print(f"  Estimated Cost: ${full_cost:.2f}")
print()



Embedding Cost Estimate:
------------------------
Average tokens per document: 1371

Finding Aids Only:
  Documents: 3,050,000
  Estimated Tokens: 4,181,550,000
  Estimated Cost: $418.16

Full Set:
  Documents: 31,600,000
  Estimated Tokens: 43,323,600,000
  Estimated Cost: $4332.36

